In [ ]:
import pandas as pd
import numpy as np
from nilearn.maskers import MultiNiftiMapsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn import datasets

# Define the directory where the data will be saved
save_directory = "D:\\ads000030"


# Fetch the dataset from openneuro with the filtering options
dataset = datasets.fetch_openneuro_dataset(urls=None, data_dir=save_directory, dataset_version='ds000030_R1.0.5', verbose=1)[source]


In [ ]:
len(dataset.func_preproc)

In [ ]:
from nilearn import datasets

# Atlas 
#setup
dim = 64
difumo = datasets.fetch_atlas_difumo(dimension=dim, resolution_mm=2, legacy_format=False)
atlas_filename = difumo.maps



# create masker using MultiNiftiMapsMasker to extract functional data within
# atlas parcels from multiple subjects using parallelization to speed up the
# # computation
masker = MultiNiftiMapsMasker(
    maps_img=atlas_filename,
    standardize=True,  # Standardize to improve comparisons between subjects
    n_jobs=-1,  # Parallelize over all available CPUs
    verbose=0
)

# ConenctivityMeasure from Nilearn uses simple 'correlation' to compute
# connectivity matrices for all subjects in a list
connectome_measure = ConnectivityMeasure(
    kind='correlation', vectorize=True, discard_diagonal=True)

In [ ]:
# Convert the dataset to a DataFrame
pheno = pd.DataFrame(dataset.phenotypic)

# Save the DataFrame as a CSV file in the correct directory
pheno.to_csv('/kaggle/working/schizo_pheno.csv', index=False)

In [ ]:
pheno.shape

In [ ]:
func_preproc_files = dataset.func_preproc

In [ ]:
# Check if the file was saved successfully
import os
print("Files in the working directory:", os.listdir('/kaggle/working/'))

In [ ]:
all_features = []
for i, sub in enumerate(func_preproc_files):
    print(f"Processing subject {i+1}...")
    
    # Step 1: Fit transform with masker
    print(f"  Fitting and transforming masker for subject {i+1}...")
    masked_data = masker.fit_transform(sub)
    
    # Step 2: Fit transform with connectome_measure
    print(f"  Fitting and transforming connectome measure for subject {i+1}...")
    transformed_data = connectome_measure.fit_transform([masked_data])[0]
    
    # Step 3: Append to all_features
    print(f"  Appending features for subject {i+1}...")
    all_features.append(transformed_data)
    
    # Print statement after completing processing for the subject
    print(f"  Completed processing for subject {i+1}\n")

print("All subjects processed.")


In [ ]:
# Let's save the data to disk

np.savez_compressed('Autism_Classf_featuress.npz',a = all_features)